In [1]:
from __future__ import absolute_import, division, print_function # Python2 compatibility
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from The_Payne import utils
from The_Payne import training

import torch
from The_Payne import spectral_model
from The_Payne import fitting
from astropy.table import Table,unique

from astropy.io import fits
from scipy.optimize import curve_fit
import scipy

from isochrones.mist.bc import MISTBolometricCorrectionGrid
from isochrones import get_ichrone, SingleStarModel

# I defined that class in the Python code of the age estimation,
# but can not import the age estimation code without defining the class here again
# Ask Matthias, how we can avoid this import problem...
class read_iso():
    def __init__(self):
        self.num_cols=4
        self.columns = ['M_Mo', 'logTeff', 'logG', 'logL_Lo']
        self.num_ages = len(age)
        self.ages = age

    def fill_chemistry(self, m_h, fe_h, alpha_fe):
        self.FeH = fe_h
        self.Z = 10**m_h*0.0152
        self.aFe = alpha_fe

    def fill_iso(self, iso_input):
        self.data = iso_input

In [2]:
#We try to import the age/mass interpolation code here,
#which is used later for the function mass_interpolation()
elli_dir = './ELLI/'
age_mass_guess_parsec = elli_dir+'age_mass_guess_parsec.py'
import sys
import os
sys.path.append(os.path.dirname(os.path.expanduser(age_mass_guess_parsec)))
try:
    
    import age_mass_guess_parsec
    # In addition, we load the Parsec isochrone data into the variable 'y'
    y = np.load(elli_dir+'Parsec_isochrones.npy',allow_pickle=True,encoding='latin1')
    # and safe their [Fe/H] values into feh_iso
    feh_iso = [i.FeH for i in y]
except:
    print('Could not find ELLI code at ',elli_dir+'Parsec_isochrones.npy')
    raise 

no sys.argv


In [3]:
wavelength = utils.load_wavelength_array(survey='galah')

#mask = np.zeros(wavelength.size, dtype=bool) # no masking
#mask = utils.galah_mask(wavelength, sme_like=True, cores_out=False)
mask = np.load("mask.npz")['mask']

NN_coeffs = utils.read_in_neural_network()

mist = get_ichrone('mist')
bc_grid = MISTBolometricCorrectionGrid(['J', 'H', 'K', 'G', 'BP', 'RP', 'g', 'r', 'i'])

In [4]:
w_array_0, w_array_1, w_array_2, b_array_0, b_array_1, b_array_2, full_x_min, full_x_max = NN_coeffs

In [5]:
try:
    galah_selection = np.load("The_Payne/other_data/galah_selection_191030.npz")     
except:
    galah_selection = np.load("galah_selection_191030.npz")

sobject_id = galah_selection['sobject_id']
flux = galah_selection['flux']
flux_error = galah_selection['flux_error']

fits = fits.open('/home/zhangxu/Payne_GALAH/4star.fits', ignore_missing_end=True)
data = fits[1].data

flux = data['flux']
flux_error = data['flux_error']

In [6]:
def optimize_logg_from_extra_input(teff, fe_h, alpha_fe, vbroad, ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, ebv, init_logg, nu_max=None, teff_sun=5772., logg_sun = 4.438, m_bol_sun = 4.7554, nu_max_sun = 3090.0, debug=False):
    def bc_interpolation(teff, init_logg, fe_h, ebv, keep_bc_ks=None, debug=False):
        
        bc_ks = bc_grid.interp([float(teff), float(init_logg), min([max([float(fe_h),-2.4]),0.6]), float(ebv)], ['K'])[0]
        print('bc:',bc_ks)
        return(bc_ks)
    
    def lbol_function(ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, bc_ks, e_bc_ks=0.05, m_bol_sun = m_bol_sun, debug=False):
           
        l_bol = 10.**(-0.4*(ks_m - 5.*np.log10(d/10.) + bc_ks - a_ks - m_bol_sun))
        
        l_bol_lo = 10.**(-0.4*(ks_m+ks_msigcom - 5.*np.log10(d_lo/10.) + (bc_ks + e_bc_ks) - np.max([0,(a_ks - e_a_ks)]) - m_bol_sun))
        l_bol_hi = 10.**(-0.4*(ks_m-ks_msigcom - 5.*np.log10(d_hi/10.) + (bc_ks - e_bc_ks) - (a_ks + e_a_ks) - m_bol_sun))
        
        e_l_bol = 0.5*(l_bol_hi - l_bol_lo)
        
        return(l_bol, e_l_bol) 
    
#     def mass_interpolation(teff, init_logg, fe_h, debug=False):
           
#         params = {'Teff': (teff, 100), 'logg': (init_logg, 0.1), 'feh': (fe_h, 0.15)}  # mas
#         mod = SingleStarModel(mist, **params)
#         mod.fit_multinest(resume=False,init_MPI=True,verbose=False)
#         mass=mod.derived_samples.mean()['mass']
#         print('mass:',mass)
#         return(mass)

    def mass_interpolation(teff, init_logg, fe_h, l_bol, e_lbol, e_teff=100, e_logg=0.5, e_fe_h=0.2, debug=False, show_todo=False):
        """
        This function estimates the mass from the given input values by calling the ELLI code
        
        Because we are fitting the parameters, we do not know their uncertainties.
        The routine needs large enough uncertainties to explore enough isochrone points.
        We are thus typically using typical uncertainties (see below)
        
        INPUT:
        teff         : sme.teff
        current_logg : current best logg estimate (will be updated iteratively)
        fe_h         : sme.fe_h
        l_bol        : bolometric luminosity
        e_lbol       : uncertainty of l_bol, with uncertainties
        e_teff       : 100 K
        e_logg       : 0.5 dex (to give it less weight as the l_bol)
        e_fe_h       : 0.2 dex (to have at least 2 isochrone [Fe/H] points)
        
        debug        : True/False
        show_todo    : True/False
        
        OUTPUT:
        mass : either isochrone-interpolated mass (to be written) or keep_mass
        """
    
#         age, mass = age_mass_guess_parsec.do_age_mass_guess(
#             np.array([teff, current_logg, l_bol, min([max([float(fe_h),-2.4]),0.6])]), 
#             np.array([e_teff, e_logg, e_lbol, e_fe_h]), 
#             y,feh_iso
#             )

        age, mass = age_mass_guess_parsec.do_age_mass_guess(
            np.array([teff, init_logg, l_bol, min([max([float(fe_h),-2.4]),0.6])]), 
            np.array([e_teff, e_logg, e_lbol, e_fe_h]), 
            y,feh_iso
            )
        print('mass:',mass)   
#         if debug:
#             print('////////')
#             print('MASS estimation:')
#             print('teff, current_logg, fe_h+0.1, l_bol, e_lbol, e_teff,e_logg,e_fe_h')
#             print("{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}".format(teff, current_logg, fe_h, l_bol, e_lbol, e_teff,e_logg,e_fe_h))
#             if (fe_h>0.6)|(fe_h<-2.4):
#                 print('adjusted fe_h because it was out of the isochrone borders')
#             print('Mass: ',mass)
#             print('Age: ',age)
#             print('////////')

        return(mass)


    def logg_parallax(mass, teff, l_bol, teff_sun=teff_sun, logg_sun = logg_sun, debug=False):
       
        return(np.log10(mass)+4.*np.log10(teff/teff_sun)-np.log10(l_bol)+logg_sun)
 
    def update_logg_parallax(teff, init_logg, fe_h, ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, ebv, debug=debug):
    
        bc_ks = bc_interpolation(teff, init_logg, fe_h, ebv, keep_bc_ks = None, debug=debug)

        l_bol, e_lbol = lbol_function(ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, bc_ks, m_bol_sun = m_bol_sun, debug=debug)
        
        mass  = mass_interpolation(teff, init_logg, fe_h, l_bol, e_lbol, debug=debug)
 
        return(logg_parallax(mass, teff, l_bol, debug=debug))
    
    
    ###blow not sub function
    if nu_max == None:
        
        updated_logg = update_logg_parallax(teff, init_logg, fe_h, ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, ebv, debug=debug)
        
        while np.abs(updated_logg - init_logg) > 0.001:
            init_logg = updated_logg
            updated_logg = update_logg_parallax(teff, init_logg, fe_h, ks_m, ks_msigcom, d, d_lo, d_hi, a_ks, e_a_ks, ebv, debug=debug)
            updated_logg = updated_logg
        
        logg=updated_logg
    else:
        # Estimate logg
        logg = logg_numax(teff, nu_max, debug=debug)

    return(logg)

#iterate two times can get logg for one star

In [7]:
def fit_normalized_spectrum_single_star_model(norm_spec, spec_err, NN_coeffs, wavelength, mask, extra_input = None, p0 = None, debug=False, **kwargs):
    
    tol = 5e-4
   
    spec_err[mask] = 999.
    print('spec_err:', spec_err)
    w_array_0, w_array_1, w_array_2, b_array_0, b_array_1, b_array_2, full_x_min, full_x_max = NN_coeffs
    
    def extra_fit_func(dummy_variable, *labels):
        
        i_teff, i_logg, i_fe_h, i_alpha_fe, i_vbroad = (np.array(labels)+0.5)*(full_x_max-full_x_min) + full_x_min #unnormalised
        print('iteff,ilogg,ifeh,ialpha:', i_teff, i_logg, i_fe_h, i_alpha_fe, i_vbroad)
        
        new_logg=optimize_logg_from_extra_input(
            teff=i_teff, fe_h=i_fe_h, alpha_fe=i_alpha_fe, vbroad=i_vbroad, 
            ks_m=extra_input['ks_m'],d=extra_input['r_est'],a_ks=extra_input['a_ks'], ebv=extra_input['ebv'],
            init_logg=extra_input['logg'], ks_msigcom=extra_input['ks_msigcom'], d_lo=extra_input['r_lo'],
            d_hi=extra_input['r_hi'], e_a_ks=extra_input['e_a_ks'],
            nu_max=None,
            debug=debug,
            **kwargs
        )

        input_labels = np.concatenate(([labels[0]],[(new_logg-full_x_min[1])/(full_x_max[1]-full_x_min[1]) - 0.5],labels[2:])) #normalised
        print('input_labels:', input_labels)
        norm_spec_from_payne = spectral_model.get_spectrum_from_neural_net(scaled_labels = input_labels, 
            NN_coeffs = NN_coeffs, wavelength = wavelength, obs_spec=norm_spec)
    
        return norm_spec_from_payne
    
    if extra_input == None:
        fit_func = simple_fit_func
        
        num_labels = w_array_0.shape[-1] + 1
    else:
        fit_func = extra_fit_func
        
        num_labels = w_array_0.shape[-1]
    
    if p0 == None:
        p0 = np.zeros(num_labels)
#     else:
#     p0 = p0
    print('p0',p0)
    # don't allow the minimimizer to go outside the range of training set
    bounds = np.zeros((2,num_labels))
    bounds[0,:] = -0.5
    bounds[1,:] = 0.5
    bounds[0,-1] = -0.5
    bounds[1,-1] = 0.5
        
    # run the optimizer
    popt, pcov = curve_fit(fit_func, xdata=[], ydata = norm_spec, sigma = spec_err, p0 = p0,
                bounds = bounds, ftol = tol, xtol = tol, absolute_sigma = True, method = 'trf')
    model_spec = fit_func([], *popt)
    print('model_spec:', model_spec)
    
    
    # rescale the results back to normal unit
    if extra_input == None:
        popt[:-1] = (popt[:-1]+0.5)*(full_x_max-full_x_min) + full_x_min
        pcov[:-1,:-1] = pcov[:-1,:-1]*(full_x_max-full_x_min)
        return(popt, pcov, model_spec)
    else:
        
        popt = (popt+0.5)*(full_x_max-full_x_min) + full_x_min
        pcov = pcov*(full_x_max-full_x_min)
        
        pop_logg = optimize_logg_from_extra_input(
            teff=popt[0], fe_h=popt[2], alpha_fe=popt[3], vbroad=popt[4], 
            ks_m=extra_input['ks_m'], d=extra_input['r_est'], a_ks=extra_input['a_ks'], ebv=extra_input['ebv'],
            init_logg=extra_input['logg'], ks_msigcom=extra_input['ks_msigcom'], d_lo=extra_input['r_lo'],
            d_hi=extra_input['r_hi'], e_a_ks=extra_input['e_a_ks'],
            nu_max=None,
            debug=debug,
            **kwargs
        )

        popt = np.concatenate(([popt[0]],[pop_logg],popt[2:]))
    
        return(popt, pcov, model_spec)

In [8]:
def fit_labels(data, debug=False, use_extra_input=True):

    fitted_labels = []
    fitted_spectra = []
    index = 0
    
#     for each in range(len(data)):
    for each in [2]:
       
        print('data:', data['teff'])
        if use_extra_input:
        
            extra_input = data[each]
#             extra_input['teff'] = 5000. 
#             extra_input['logg'] = 3.5
#             extra_input['fe_h'] = 0.
#             extra_input['alpha_fe'] = 0.
#             extra_input['vbroad'] = 8.
            
#             init_input = np.array([min([max([float(extra_input['teff']),4000]), 6750]), 
#                                     min([max([float(extra_input['logg']),-0.5]), 5.0]), 
#                                     min([max([float(extra_input['fe_h']),-0.75]), 1.0]), 
#                                     min([max([float(extra_input['alpha_fe']),-0.4]), 0.8]), 
#                                     min([max([float(extra_input['vbroad']), 7.0]), 35.0])])

#             norm_init_input = (init_input-full_x_min)/(full_x_max-full_x_min) - 0.5
            
#             index=index+1
#             print('index', index, norm_init_input)
                
        else:
            extra_input = None
        
        popt, pcov, best_fit_spec = fit_normalized_spectrum_single_star_model(
            norm_spec = flux[each], 
            spec_err = flux_error[each],
            NN_coeffs = NN_coeffs,
            wavelength=wavelength,
            mask=mask,
            extra_input=extra_input,
            p0=None,
            debug=debug 
        )

        if debug==True:
            f, gs = plt.subplots(4,figsize=(15,5))
            for each_ccd in range(4):
                i_wave = (wavelength > (4+each_ccd)*1000) & (wavelength < (5+each_ccd)*1000)
                ax = gs[each_ccd]
                ax.plot(wavelength[i_wave],flux[each][i_wave])
                ax.plot(wavelength[i_wave],best_fit_spec[i_wave])
                ax.set_ylim(0,1.1)
            plt.tight_layout()

        # append the best fit parameters to "fitted_labels"
        print('fitted_labels:', popt)
        fitted_labels.append(popt)
        fitted_spectra.append(best_fit_spec)
        
    return(np.array(fitted_labels), np.array(fitted_spectra))

fitted_labels, fitted_spectra = fit_labels(data, debug=False)

data: [4289.1626 5797.452  5778.8574 6467.9033]
spec_err: [9.99000000e+02 2.14040659e-03 2.16663513e-03 ... 9.99000000e+02
 9.99000000e+02 2.34927325e-03]
p0 [0. 0. 0. 0. 0.]
iteff,ilogg,ifeh,ialpha: 5500.0 2.25 -1.0 0.20000001788139343 21.0
bc: 1.5697495708961486
mass: 0.7594610079735283
bc: 1.5695605103029333
mass: 0.7595025445095189
input_labels: [0.         0.36697071 0.         0.         0.        ]
kai2 spectrum and obs_spec: Power_divergenceResult(statistic=46.15386593771342, pvalue=1.0)
fit: [ 4.80867162e-04 -1.23756132e+00]
fit: [1.00815144e-04 5.50179521e-01]
fit: [ 7.53775298e-04 -2.58882250e+00]
fit: [-2.67180287e-04  2.31392568e+00]
fit: [-4.28559865e-05  1.24139852e+00]
fit: [-8.03151747e-04  5.56348406e+00]
fit: [ 3.64562491e-04 -1.08080440e+00]
fit: [1.08521092e-04 3.98971741e-01]
fit: [-4.40514690e-04  3.55058206e+00]
fit: [ 1.49021619e-03 -7.71876669e+00]
fit: [-1.91658847e-04  2.25136015e+00]
fit: [ 2.27287309e-04 -4.71431386e-01]
fit: [-1.53437769e-04  2.02025437e+

fit: [ 4.80111475e-04 -1.23399126e+00]
fit: [1.03721109e-04 5.36358730e-01]
fit: [ 7.54658704e-04 -2.59305100e+00]
fit: [-2.67662258e-04  2.31624645e+00]
fit: [-4.08989796e-05  1.23189262e+00]
fit: [-8.02954167e-04  5.56236444e+00]
fit: [ 3.63447041e-04 -1.07441197e+00]
fit: [1.06296921e-04 4.11815194e-01]
fit: [-4.40803625e-04  3.55226185e+00]
fit: [ 1.49103846e-03 -7.72358231e+00]
fit: [-1.92299247e-04  2.25551426e+00]
fit: [ 2.27759546e-04 -4.74540871e-01]
fit: [-1.53078049e-04  2.01787394e+00]
fit: [8.69919263e-05 4.17631401e-01]
fit: [ 2.16795723e-04 -4.46538141e-01]
fit: [ 2.25441663e-04 -7.54054819e-01]
fit: [ 0.00014765 -0.14763226]
fit: [ 1.65061000e-04 -2.91425039e-01]
kai2 renormalise_spectrum and obs_spec: Power_divergenceResult(statistic=34.00310498757298, pvalue=1.0)
iteff,ilogg,ifeh,ialpha: 5499.999629145403 2.2499989228919905 -1.0000008822885564 0.20000001768108067 21.00000053306318
bc: 1.56974975659389
mass: 0.7594608790447502
bc: 1.5695606974807248
mass: 0.75950241563

fit: [ 4.85622697e-04 -1.26002762e+00]
fit: [9.84041182e-05 5.61643659e-01]
fit: [ 7.52072293e-04 -2.58067082e+00]
fit: [-2.66246266e-04  2.30942831e+00]
fit: [-3.95062739e-05  1.22512755e+00]
fit: [-8.03683119e-04  5.56649517e+00]
fit: [ 3.64584024e-04 -1.08092780e+00]
fit: [1.08469571e-04 3.99268579e-01]
fit: [-4.40485979e-04  3.55041515e+00]
fit: [ 1.49039977e-03 -7.71984179e+00]
fit: [-1.92269100e-04  2.25531868e+00]
fit: [ 2.28318299e-04 -4.78218455e-01]
fit: [-1.53203658e-04  2.01870515e+00]
fit: [8.73932901e-05 4.14956401e-01]
fit: [ 2.17305761e-04 -4.49965691e-01]
fit: [ 2.2544356e-04 -7.5406950e-01]
fit: [ 1.47680416e-04 -1.47883953e-01]
fit: [ 1.64640036e-04 -2.88111430e-01]
kai2 renormalise_spectrum and obs_spec: Power_divergenceResult(statistic=34.00182450853686, pvalue=1.0)
iteff,ilogg,ifeh,ialpha: 5499.999629145403 2.2499989228919905 -1.0000008822885564 0.20000001768108067 21.00000053306318
bc: 1.56974975659389
mass: 0.7594608790447502
bc: 1.5695606974807248
mass: 0.75950

In [ ]:
# 3 解决data会变的问题

In [ ]:
print('fitted_spectra:','\n', fitted_spectra)
print('obs_spec:', '\n', data['flux'])
print('fitted_labels:Arcturus, betHyi, Sun, Procyon', '\n', fitted_labels.T)
print('GALAH DR3 values:','\n', data['teff'],'\n', data['logg'], '\n', data['fe_h'], '\n', data['alpha_fe'], '\n', data['vbroad'])

In [ ]:
from astropy.io import fits
fits = fits.open('/home/zhangxu/Payne_GALAH/4star.fits', ignore_missing_end=True)
data = fits[1].data
TEFF = data['teff']
LOGG = data['logg']
FEH = data['feh']
ALPHA = data['alpha_fe']
teff = fitted_labels.T[0,:]
logg = fitted_labels.T[1,:]
feh = fitted_labels.T[2,:]
alpha = fitted_labels.T[3,:]

In [ ]:
plt.scatter(TEFF,LOGG,c='r')
plt.scatter(teff,logg)
plt.xlim(7000,3500)
plt.ylim(5.5,0)

In [ ]:
plt.scatter(FEH,ALPHA,c='r')
plt.scatter(feh,alpha)

In [ ]:
synthetic_SunArc = np.load("GALAH_NordlanderSunArc_GUESSnorm.npz")
smod_norm_guess = synthetic_SunArc['smod_norm_guess']
interpolated_sun_spectra = np.array([scipy.interp(wavelength, synthetic_SunArc['wavelength'], smod_norm_guess[1])]).reshape(14304)
interpolated_arc_spectra = np.array([scipy.interp(wavelength, synthetic_SunArc['wavelength'], smod_norm_guess[0])]).reshape(14304)

In [ ]:
real_labels_4 = [5772, 4.438, 0.0, 0.0, 15]

scaled_labels_4 = (real_labels_4[:]-full_x_min)/(full_x_max-full_x_min) - 0.5
real_spec_4 = spectral_model.get_spectrum_from_neural_net(scaled_labels = scaled_labels_4, NN_coeffs = NN_coeffs, wavelength=wavelength, obs_spec=flux[2,:])

In [ ]:
fitted_spectra

In [ ]:
ccd = dict()
for each_ccd in range(4):
    ccd[each_ccd] = (wavelength > 1000*(4+each_ccd)) & (wavelength < 1000*(5+each_ccd))

f, gs = plt.subplots(4,1,figsize=(15,10))
for it in range(4):
    ax=gs[it]

    ax.plot(wavelength[ccd[it]], real_spec_4[ccd[it]], 'g', lw=0.5)                     #from ture value
    ax.plot(wavelength[ccd[it]], flux[2,:][ccd[it]], 'b', lw=0.5)                           #obs spectra
#    ax.plot(wavelength[ccd[it]], fitted_spectra[0][ccd[it]], 'y', lw=0.5)                  #Payne best fitted spectra
#    ax.plot(wavelength[ccd[it]], interpolated_sun_spectra[ccd[it]], 'r', lw=0.5)            #synthetic spectra
#    ax.scatter(wavelength[ccd[it]&mask], flux[2,:][ccd[it]&mask], s=1, c='g', marker='o')   #mask dots

#    ax.plot(wavelength[ccd[it]], flux[2,:][ccd[it]]-interpolated_sun_spectra[ccd[it]], 'g', lw=0.5)
    ax.plot(wavelength[ccd[it]], flux[2,:][ccd[it]]-real_spec_4[ccd[it]], 'r', lw=0.5)
#    ax.scatter(wavelength[ccd[it]&mask], flux[2,:][ccd[it]&mask]-interpolated_sun_spectra[ccd[it]&mask], s=0.1, c='r', marker='o')

#    ax.legend(labels=['Observed Sun spec','Synthetic spec from Thomas'],loc='best')
    ax.set_ylabel('Flux',fontsize=15)
fig = plt.gcf()
fig.savefig('Sun_spectra.png', format='png')

Use synthetic spectra of Sun and Arc form Norlander to check model we trained

In [ ]:
# synthetic_SunArc = np.load("GALAH_NordlanderSunArc_GUESSnorm.npz")
# smod_norm_guess = synthetic_SunArc['smod_norm_guess']
# interpolated_sun_spectra = np.array([scipy.interp(wavelength, synthetic_SunArc['wavelength'], smod_norm_guess[1])]).reshape(14304)
# interpolated_arc_spectra = np.array([scipy.interp(wavelength, synthetic_SunArc['wavelength'], smod_norm_guess[0])]).reshape(14304)

In [ ]:
# #for sun
# spec = interpolated_sun_spectra
# spec_err = interpolated_sun_spectra*0.01

# popt, pcov, best_fit_spec = fitting.fit_normalized_spectrum_single_star_model(norm_spec = spec, 
#         spec_err = spec_err, NN_coeffs = NN_coeffs, wavelength=wavelength, mask=mask, p0 = None)
# print('sun:',popt)

# kai = scipy.stats.chisquare(f_obs=best_fit_spec, f_exp=interpolated_sun_spectra)
# print('kai^2 between payne best fitted spectrum and Thomas synthetic spectra of sun:',kai[0])

# #for Arcturus
# spec = interpolated_arc_spectra
# spec_err = interpolated_arc_spectra*0.01

# popt, pcov, best_fit_spec = fitting.fit_normalized_spectrum_single_star_model(norm_spec = spec, 
#         spec_err = spec_err, NN_coeffs = NN_coeffs, wavelength=wavelength, mask=mask, p0 = None)
# print('Arc',popt)

# kai = scipy.stats.chisquare(f_obs=best_fit_spec, f_exp=interpolated_arc_spectra)
# print('kai^2 between payne best fitted spectrum and Thomas synthetic spectra of Arcturus:', kai[0])